In [0]:
%python
"Restoring serverless notebook session state..."

In [0]:
%sql
CREATE EXTERNAL VOLUME IF NOT EXISTS stock_pipeline.default.bronze_volume
LOCATION 's3://stock-pipeline-data-dev-mc/bronze/';



In [0]:
%pip install --force-reinstall --no-cache-dir /Volumes/stock_pipeline/default/wheel_volume/stock_pipeline-0.1.dev65-py3-none-any.whl

In [0]:
dbutils.library.restartPython()

In [0]:
display(spark.createDataFrame([(k,) for k in dbutils.secrets.list("fmp_scope")], ["key"]))

In [0]:
import asyncio
from datetime import datetime, timedelta
from bronze.utils import AsyncFMPClient, FMPConfig, create_spark_session
from bronze.ingestion.fmp import ingest_ticker_bronze
from bronze.ingestion.schemas import get_bronze_schemas

# Set up your environment
import os
os.environ['FMP_API_KEY'] = dbutils.secrets.get("fmp_scope", "FMP_API_KEY")
os.environ['S3_BUCKET_BRONZE'] = dbutils.secrets.get("fmp_scope", "S3_BUCKET_BRONZE")
os.environ['AWS_ACCESS_KEY_ID'] = dbutils.secrets.get("fmp_scope", "AWS_ACCESS_KEY_ID")
os.environ['AWS_SECRET_ACCESS_KEY'] = dbutils.secrets.get("fmp_scope", "AWS_SECRET_ACCESS_KEY")

async def test_ingestion():
    # Initialize components (same as main() does)
    config = FMPConfig(api_key=os.environ["FMP_API_KEY"])
    client = AsyncFMPClient(config)
    spark = create_spark_session()
    schemas = get_bronze_schemas()

    # Define parameters (equivalent to --tickers AAPL --backfill)
    tickers = ["AAPL", "MSFT"]
    backfill = False  # Set to True for 5-year history

    # Calculate date range
    if backfill:
        to_date = datetime.now().strftime("%Y-%m-%d")
        from_date = (datetime.now() - timedelta(days=5*365)).strftime("%Y-%m-%d")      
        print(f"📅 Backfill mode: {from_date} to {to_date}")
    else:
        yesterday = datetime.now() - timedelta(days=1)
        from_date = to_date = yesterday.strftime("%Y-%m-%d")
        print(f"📅 Daily mode: {from_date}")

    # Process tickers
    results = []
    for ticker in tickers:
        print(f"🔄 Processing {ticker}...")
        result = await ingest_ticker_bronze(client, spark, ticker, from_date if from_date else '', to_date if to_date else '', schemas)
        results.append(result)
        print(f"✅ {ticker} complete: {result['record_counts']}")

    spark.stop()
    return results

# Run the test
results = await test_ingestion()
print(f"🎯 Ingestion complete: {len(results)} tickers processed")



